# 📊 Module 4: Generate Reports for Data and Model Drift

In this module, we'll use [Evidently](https://evidentlyai.com/) to generate reports that help monitor and detect:

- **Model Drift**
- **Data Drift**
- **Target Drift**

We'll start by loading **Reference Data** and **Current Data**.

This simulates comparing a baseline dataset against new incoming production data.

In [1]:
# Install requirements
!pip install -r requirements.txt


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


## 📦 Import Required Libraries

Before we proceed with training and tracking our machine learning model, we need to import the necessary libraries.


In [27]:
# Import necessary modules
import os
import joblib
import requests

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

import pandas as pd
import numpy as np

from evidently import ColumnMapping
from evidently.report import Report
from evidently.metric_preset import DataDriftPreset, TargetDriftPreset, RegressionPreset
from evidently.metric_preset import DataQualityPreset
from evidently.metric_preset import RegressionPreset

import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient

## 📁 Load Reference and Current Data

In [ ]:
# Load reference datasets (January & February)
data_path = "./data/processed/"

data_01 = pd.read_csv(data_path + 'data_2011_01.csv')
data_02 = pd.read_csv(data_path + 'data_2011_02.csv')

reference_data = pd.concat([data_01, data_02], ignore_index=True)

# Load current dataset (March) 
current_data_source = "data_2011_03"
current_data = pd.read_csv(data_path + f"{current_data_source}.csv")

# Preview shapes and basic info
print("Reference data shape:", reference_data.shape)
print("Current data shape:", current_data.shape)
# reference_data.head()

Reference data shape: (1337, 17)
Current data shape: (730, 17)


## Batch Inference: Sending Multiple Requests to the Model Endpoint on OpenShift

This section reads multiple JSON-formatted input samples from a file and sends them one-by-one to the model's prediction endpoint. It collects the predicted results and displays them alongside the actual target values in a combined output DataFrame.

### Get prediciton on Reference Data

In [45]:
# Define the subset of features expected by the model
model_features = [
    'temp', 'atemp', 'humidity', 'windspeed',
    'hour', 'weekday', 'season', 'holiday', 'workingday', 'weathersit'
]

# Extract the relevant subset and convert to JSON-like list of dicts
json_input_list_reference = reference_data[model_features].to_dict(orient='records')

print(json_input_list_reference[:5])

[{'temp': 0.24, 'atemp': 0.2879, 'humidity': 0.81, 'windspeed': 0.0, 'hour': 0, 'weekday': 6, 'season': 1, 'holiday': 0, 'workingday': 0, 'weathersit': 1}, {'temp': 0.22, 'atemp': 0.2727, 'humidity': 0.8, 'windspeed': 0.0, 'hour': 1, 'weekday': 6, 'season': 1, 'holiday': 0, 'workingday': 0, 'weathersit': 1}, {'temp': 0.22, 'atemp': 0.2727, 'humidity': 0.8, 'windspeed': 0.0, 'hour': 2, 'weekday': 6, 'season': 1, 'holiday': 0, 'workingday': 0, 'weathersit': 1}, {'temp': 0.24, 'atemp': 0.2879, 'humidity': 0.75, 'windspeed': 0.0, 'hour': 3, 'weekday': 6, 'season': 1, 'holiday': 0, 'workingday': 0, 'weathersit': 1}, {'temp': 0.24, 'atemp': 0.2879, 'humidity': 0.75, 'windspeed': 0.0, 'hour': 4, 'weekday': 6, 'season': 1, 'holiday': 0, 'workingday': 0, 'weathersit': 1}]


In [ ]:
# ── Endpoint ─────────────────────────────────────────────────────────────
endpoint = "http://MODEL_API_SERVIC:80/predict"

# ── Read the JSON-line file and send requests ────────────────────────────
ref_predictions = []

for item in json_input_list_reference:    
    inference_request = item
    response = requests.post(endpoint, json=inference_request)
    
    if response.status_code == 200:
        # FastAPI returns {"prediction": <value>}
        ref_predictions.append(response.json()["prediction"])
    
    else:
        print(f"Request failed with status code: {response.status_code}")
        print(f"Response content: {response.text}")

# ── Assemble results into a DataFrame ────────────────────────────────────
ref_pred_df = pd.DataFrame(ref_predictions, columns=["Predicted Count"])

print(ref_pred_df.head())

   Predicted Count
0            23.63
1            35.44
2            24.74
3            11.97
4             1.67


In [47]:
reference_data["prediction"] = ref_pred_df
reference_data.head()

,dteday,instant,season,year,month,hour,holiday,weekday,workingday,weathersit,temp,atemp,humidity,windspeed,casual,registered,count,prediction
0,2011-01-01,1,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16,23.63
1,2011-01-01,2,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40,35.44
2,2011-01-01,3,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32,24.74
3,2011-01-01,4,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13,11.97
4,2011-01-01,5,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1,1.67


### Get prediciton on Current Data

In [48]:
# Define the subset of features expected by the model
model_features = [
    'temp', 'atemp', 'humidity', 'windspeed',
    'hour', 'weekday', 'season', 'holiday', 'workingday', 'weathersit'
]

# Extract the relevant subset and convert to JSON-like list of dicts
json_input_list_current = current_data[model_features].to_dict(orient='records')

print(json_input_list_current[:5])

[{'temp': 0.3, 'atemp': 0.2727, 'humidity': 0.7, 'windspeed': 0.4627, 'hour': 0, 'weekday': 2, 'season': 1, 'holiday': 0, 'workingday': 1, 'weathersit': 1}, {'temp': 0.26, 'atemp': 0.2273, 'humidity': 0.7, 'windspeed': 0.3582, 'hour': 1, 'weekday': 2, 'season': 1, 'holiday': 0, 'workingday': 1, 'weathersit': 1}, {'temp': 0.24, 'atemp': 0.2121, 'humidity': 0.65, 'windspeed': 0.3881, 'hour': 2, 'weekday': 2, 'season': 1, 'holiday': 0, 'workingday': 1, 'weathersit': 1}, {'temp': 0.22, 'atemp': 0.2121, 'humidity': 0.69, 'windspeed': 0.2836, 'hour': 3, 'weekday': 2, 'season': 1, 'holiday': 0, 'workingday': 1, 'weathersit': 1}, {'temp': 0.22, 'atemp': 0.2121, 'humidity': 0.69, 'windspeed': 0.2537, 'hour': 4, 'weekday': 2, 'season': 1, 'holiday': 0, 'workingday': 1, 'weathersit': 1}]


In [ ]:
# ── Endpoint ─────────────────────────────────────────────────────────────
endpoint = "http://MODEL_API_SERVIC:80/predict"

# ── Read the JSON-line file and send requests ────────────────────────────
cur_predictions = []

for item in json_input_list_current:    
    inference_request = item
    response = requests.post(endpoint, json=inference_request)
    
    if response.status_code == 200:
        # FastAPI returns {"prediction": <value>}
        cur_predictions.append(response.json()["prediction"])
    
    else:
        print(f"Request failed with status code: {response.status_code}")
        print(f"Response content: {response.text}")

# ── Assemble results into a DataFrame ────────────────────────────────────
cur_pred_df = pd.DataFrame(cur_predictions, columns=["Predicted Count"])

print(cur_pred_df.head())

   Predicted Count
0        16.400000
1         6.020000
2         2.719751
3         2.010547
4         1.816456


In [50]:
current_data["prediction"] = cur_pred_df
current_data.head()

,dteday,instant,season,year,month,hour,holiday,weekday,workingday,weathersit,temp,atemp,humidity,windspeed,casual,registered,count,prediction
0,2011-03-01,1338,1,0,3,0,0,2,1,1,0.30,0.2727,0.70,0.4627,0,7,7,16.400000
1,2011-03-01,1339,1,0,3,1,0,2,1,1,0.26,0.2273,0.70,0.3582,0,3,3,6.020000
2,2011-03-01,1340,1,0,3,2,0,2,1,1,0.24,0.2121,0.65,0.3881,0,4,4,2.719751
3,2011-03-01,1341,1,0,3,3,0,2,1,1,0.22,0.2121,0.69,0.2836,0,2,2,2.010547
4,2011-03-01,1342,1,0,3,4,0,2,1,1,0.22,0.2121,0.69,0.2537,0,1,1,1.816456


## 🗺️ Define Column Mapping

Evidently supports specifying column roles explicitly via `ColumnMapping`, which helps produce more accurate and meaningful metrics.

Here, we define:
- `target`: the actual value to predict (`count`)
- `prediction`: (optional) placeholder for model prediction column
- `numerical_features`: continuous input features
- `categorical_features`: categorical or discrete input features


In [51]:
target="count"
prediction="prediction"
numerical_features=['temp', 'atemp', 'humidity', 'windspeed', 'hour', 'weekday']
categorical_features=['holiday', 'workingday', 'weathersit']

# For now, we ignore season for data drift report,
# since it does not affect the conclusion
# categorical_features=['season', 'holiday', 'workingday', 'weathersit']

column_mapping = ColumnMapping(
    target="count",
    prediction="prediction",
    numerical_features=['temp', 'atemp', 'humidity', 'windspeed', 'hour', 'weekday'],
    categorical_features=['holiday', 'workingday', 'weathersit']
    )
# column_mapping.target = target
# column_mapping.prediction = prediction
# column_mapping.numerical_features = numerical_features
# column_mapping.categorical_features = categorical_features

## 📈 Generate a Regression Performance Report

The **Regression Performance Report** evaluates how well a model performs over time.

To simulate production monitoring, we'll assume that a `prediction` column already exists in the dataset (this could be added via an inference pipeline). The report will compare the predicted and actual target values (`count`) and show metrics like:
- RMSE
- R²
- Error distribution
- Prediction quality


In [11]:
# Create the Regression Performance report
regression_report = Report(metrics=[RegressionPreset()])

# Run the report with column mapping
regression_report.run(
    reference_data=reference_data,
    current_data=current_data,
    column_mapping=column_mapping
)

# Save the report as HTML
os.makedirs("./reports", exist_ok=True)
os.makedirs(f"./reports/{current_data_source}", exist_ok=True)
output_path = f"./reports/{current_data_source}/regression_performance_report.html"
regression_report.save_html(output_path)

print(f"✅ Regression Performance report saved to {output_path}")

/opt/app-root/lib64/python3.11/site-packages/sklearn/metrics/_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


✅ Regression Performance report saved to ./reports/2011_04/regression_performance_report.html


## 📉 Generate a Data Drift Report

We'll use the `DataDriftReport` class from Evidently to compare feature distributions between the reference (January) and current (February) datasets.

This report will help us understand whether any input features have changed significantly, which may impact model predictions.


In [52]:
# Create a report with the Data Drift preset
data_drift_report = Report(metrics=[DataDriftPreset()])

# Run the comparison
data_drift_report.run(
        reference_data=reference_data[numerical_features + categorical_features], 
        current_data=current_data[numerical_features + categorical_features], 
        column_mapping=column_mapping
    )

# Create directories if they don't exist
# report_dir = "./reports"
# os.makedirs(report_dir, exist_ok=True)

# data_drift_report.show()

# Save the report as an HTML file
output_path = f"./reports/{current_data_source}/data_drift_report.html"
data_drift_report.save_html(output_path)

print(f"✅ Data Drift report saved to {output_path}")

✅ Data Drift report saved to ./reports/2011_03/data_drift_report.html


## 🎯 Generate a Target Drift Report

We'll now generate a **Target Drift Report** using Evidently.

This report focuses specifically on changes in the distribution of the **target variable** (`cnt`), which represents the total number of bike rentals. Drift in the target distribution can indicate seasonal or behavioral changes in users that may affect model performance.


In [13]:
# Create the Target Drift report
target_drift_report = Report(metrics=[TargetDriftPreset()])

# Run the report with column mapping
target_drift_report.run(
    reference_data=reference_data,
    current_data=current_data,
    column_mapping=column_mapping
)

# Create directories if they don't exist
# report_dir = "./reports"
# os.makedirs(report_dir, exist_ok=True)

# Save the report as HTML
output_path = f"./reports/{current_data_source}/target_drift_report.html"
target_drift_report.save_html(output_path)

print(f"✅ Target Drift report saved to {output_path}")

✅ Target Drift report saved to ./reports/2011_04/target_drift_report.html


# ✅ Summary

In this module, we learned how to use Evidently to monitor data and model performance over time.

We completed the following steps:
- ✅ Evaluated **Regression Model Performance** using simulated predictions
- ✅ Compared new dataset (March) with training data (January & February) to detect **Data Drift**
- ✅ Analyzed changes in the **Target variable** distribution

These reports can be integrated into automated pipelines to continuously track the health of machine learning systems in production.


## 🧳 Select and Load a trained Model Version from MLflow

In this step, we interact with the MLflow Model Registry to:

1. **List all available versions** of a registered model (`BikeSharingModel`) along with their metadata, such as version number, stage, and run ID.
2. **Prompt the user** to choose a specific version to use for deployment or analysis.
3. **Load the selected model** from the MLflow tracking server using the model URI.

This makes it easy to manage multiple iterations of a model and ensures reproducibility when deploying or testing specific versions.


## import mlflow
from mlflow.tracking import MlflowClient

# Initialize MLflow client
MLFLOW_TRACKING_URI = 'MLFLOW_REMOTE_TRACKING_SERVER'
mlflow.set_tracking_uri(f"{MLFLOW_TRACKING_URI}")
client = MlflowClient()

model_name = "BikeSharingModel"

# List available versions
versions = client.search_model_versions(filter_string=f"name='{model_name}'", order_by=["version_number DESC"])

print("📦 Available versions for model:", model_name)
for v in versions:
    print(f"Version: {v.version}, Stage: {v.current_stage}, Status: {v.status}, Run ID: {v.run_id}")

# Ask the user to select a version
selected_version = input("Enter the version number you want to download: ").strip()

# Load the selected model version
model_uri = f"models:/{model_name}/{selected_version}"
model = mlflow.pyfunc.load_model(model_uri=model_uri)

print(f"✅ Model version {selected_version} loaded successfully from MLflow.")

In [ ]:
reference_data["prediction"] = model.predict(reference_data[numerical_features + categorical_features])
current_data["prediction"] = model.predict(current_data[numerical_features + categorical_features])
reference_data.head()

## 🧪 Generate a Data Quality Report

This report helps identify common data issues such as:
- Missing values
- Unexpected or invalid values
- Type mismatches
- Constant or duplicate columns

This is useful for ensuring the data pipeline remains clean and reliable over time.
``


In [ ]:
# Create the Data Quality report
data_quality_report = Report(metrics=[DataQualityPreset()])

# Run the report with column mapping
data_quality_report.run(
    reference_data=reference_data,
    current_data=current_data,
    column_mapping=column_mapping
)

# Save the report as HTML
output_path = "./reports/data_quality_report.html"
data_quality_report.save_html(output_path)

print(f"✅ Data Quality report saved to {output_path}")
